In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import psutil
from tqdm import tqdm_notebook
import cv2
import pickle
import time
import csv
np.random.seed(0)

In [4]:
LANG = "hindi"
IMAGE_FOLDER = "data/{}/Image".format(LANG)
ANNOTATION_FOLDER = "data/{}/Annotation".format(LANG)
HEIGHT, WIDTH = 512, 512

In [21]:
os.mkdir("data/{}/images".format(LANG))
os.mkdir("data/{}/annotations".format(LANG))

In [24]:
NEW_IMAGE_FOLDER = "data/{}/images".format(LANG)
NEW_ANNOTATION_FOLDER = "data/{}/annotations".format(LANG)

In [3]:
print(psutil.cpu_percent())
print(psutil.virtual_memory()) 

0.0
svmem(total=31628320768, available=30612586496, percent=3.2, used=542887936, free=22986735616, active=1144991744, inactive=6378262528, buffers=295591936, cached=7803105280, shared=11419648, slab=936087552)


In [29]:
order = [0, 4, 1, 5, 2, 6, 3, 7]
for sub_folder in tqdm_notebook(os.listdir(IMAGE_FOLDER)):
    for image_file_name in tqdm_notebook(os.listdir(os.path.join(IMAGE_FOLDER, sub_folder))):
        key = os.path.join(sub_folder, image_file_name)
        with Image.open(os.path.join(IMAGE_FOLDER, sub_folder, image_file_name)) as pil_image:
            old_width, old_height = pil_image.size
            w_ratio = WIDTH / old_width
            h_ratio = HEIGHT / old_height
            pil_image = pil_image.resize((WIDTH, HEIGHT), Image.ANTIALIAS)
            src = os.path.join(IMAGE_FOLDER, sub_folder, image_file_name)
            dst = os.path.join(NEW_IMAGE_FOLDER, str(sub_folder)+"_"+image_file_name)
            #pil_image.save(src)
            #shutil.move(src, dst)
            pil_image.save(dst)
        annotation_file_name = image_file_name[:-3]+"txt"
        with open(os.path.join(ANNOTATION_FOLDER, sub_folder, annotation_file_name)) as annotation_file:
            lines = annotation_file.readlines()
            words = [line.split(" ")[-1].rstrip() for line in lines]
            lines = [line.split(" ")[:-1] for line in lines]
            lines = np.array(lines).astype(np.float)
            lines[:,0:4] = w_ratio * lines[:,0:4]
            lines[:,4:8] = h_ratio * lines[:,4:8]
            lines = lines.astype(np.int)
            lines = lines[:,order]
            lines = lines.tolist()
            lines = pd.DataFrame(lines)
            lines.insert(8, "text", words)
            src = os.path.join(ANNOTATION_FOLDER, sub_folder, annotation_file_name)
            dst = os.path.join(NEW_ANNOTATION_FOLDER, str(sub_folder)+"_"+annotation_file_name)[:-3] + "csv"
            lines.to_csv(dst, index=False, header=False)

In [30]:
print(len(os.listdir("data/hindi/images")))
print(len(os.listdir("data/hindi/annotations")))

116132
116132


In [ ]:
shutil.rmtree(IMAGE_FOLDER)
shutil.rmtree(ANNOTATION_FOLDER)

In [33]:
os.remove("data/{}/readme.txt".format(LANG))